In [92]:

import numpy as np
import datetime

def time2datetime(time):
    dt = datetime(time[0],time[1],time[2],time[3],time[4],time[5])
    return dt


def find_near_time(obstime,lines):
    # 找到最接近的观测时间
    diff_time=[]
    time=[]
    for line in lines:
        if(line[0:2].strip().isdigit()):
            toc=[None] * 6
            toc[0]=int(line[3:5].strip())
            toc[1]=int(line[6:8].strip())
            toc[2]=int(line[9:11].strip())
            toc[3]=int (line[12:14].strip())
            toc[4]=int(line[15:17].strip())
            toc[5]= int(float(line[18:22].strip()))
            # obstime.astype(int)
            obs_week,obs_sec = ctime2gps(obstime) 
            toc_week,toc_sec=ctime2gps(toc)
            diff=abs((obs_week-toc_week)*604800+(obs_sec-toc_sec))
            diff_time.append(diff)
            time.append(toc)
        else:
            continue
    min_index = diff_time.index(min(diff_time))
    return time[min_index]
    
def ctime2gps(time):
    # 计算时间
    gps_epoch = datetime.datetime(1980, 1, 6)
    given_time = datetime.datetime(time[0],time[1],time[2],time[3],time[4],int(time[5]))
    time_diff = given_time - gps_epoch
    total_seconds = time_diff.total_seconds()
    gps_weeks, gps_seconds = divmod(total_seconds, 604800)
    return gps_weeks,gps_seconds

def caculate_pos_of_sat(matrix,a3,obstime,reftime):
    # matrix,a3为各种参数
    # obstime为观测时刻
    # reftime为参考时刻

    # 计算卫星的平均角速度
    GM=3.986005*pow(10,5)
    n0=np.sqrt(GM)/pow(matrix[1,3],3)
    n=n0+matrix[0,2]

    # 计算归化时间
    gps_obsweeks,gps_obssec=ctime2gps(obstime)
    gps_refweek,gps_refsec=ctime2gps(reftime)
    delta_t=a3[0]+a3[1]*(gps_obssec-gps_refsec)+a3[2]*pow(gps_obssec-gps_refsec,2)
    t=gps_refsec-delta_t
    tk=t-matrix[2,0]

    # 计算平近角点
    Mk=matrix[0,3]+n*tk

    # 计算偏近点角
    # 此处采用迭代法计算偏近点角
    Mk=Mk%(np.pi*2)
    Ek=Mk
    Ek_old=Ek
    for i in range(10):
        Ek=Mk+matrix[1,1]*np.sin(Ek)
        if(abs(Ek-Ek_old)<1e-10):
            break
        else:
            Ek_old=Ek
    # 计算真近点角
    cos_Vk=(np.cos(Ek)-matrix[1,1])/(1-matrix[1,1]*np.cos(Ek))
    sin_Vk=(np.sqrt(1-matrix[1,1]*matrix[1,1])*np.sin(Ek))/(1-matrix[1,1]*np.cos(Ek))
    Vk=np.arctan(np.sqrt(1-matrix[1,1]*matrix[1,1])*np.sin(Ek)/(np.cos(Ek)-matrix[1,1]))
    
    # 计算升交距角
    phi_k=Vk+matrix[3,2]
    phi_k=phi_k%(np.pi*2)

    # 计算摄动改正
    delta_u=matrix[1,0]*np.cos(2*phi_k)+matrix[1,2]*np.sin(2*phi_k)
    delta_r=matrix[3,1]*np.cos(2*phi_k)+matrix[0,1]*np.sin(2*phi_k)
    delta_i=matrix[2,1]*np.cos(2*phi_k)+matrix[2,3]*np.sin(2*phi_k)

    # 计算升距交角，卫星矢径，轨道倾角
    uk=phi_k+delta_u+delta_u
    rk=pow(matrix[1,3],2)*(1-matrix[1,1]*np.cos(Ek))+delta_r
    ik=matrix[3,0]+delta_i+matrix[4,0]*tk

    # 计算卫星在轨道平面坐标系的坐标
    xk=rk*np.cos(uk)
    yk=rk*np.sin(uk)

    # 计算观测时刻的升交点经度L
    we=7.29211567*1e-5
    omegak=matrix[2,2]+(matrix[3,3]-we)*tk-we*matrix[2,0]

    # 计算卫星在地固坐标系中的坐标
    Xk=-(xk*np.cos(omegak)-yk*np.cos(ik)*np.sin(omegak))
    Yk=-(xk*np.sin(omegak)+yk*np.cos(ik)*np.cos(omegak))
    Zk=-(yk*np.sin(ik))

    return Xk,Yk,Zk,delta_t




In [93]:

def caculate_sat(obstime):
            # 读取广播星历，n文件
            # 指定文件路径
            file_path = '../data/al2h3340.23n'

            with open(file_path, 'r') as file:
                lines = file.readlines()

            time=find_near_time(obstime,lines)


            target_string='END OF HEADER'
            HeaderLine=0

            for i, line in enumerate(lines, start=1):
                if target_string in line:
                    HeaderLine=i
                    break
            # 观测值默认为GPS
            obs_type='GPS'
            if('GPS'in lines[0]):
                obs_type='GPS'

            # 设定卫星编号
            # satlite_PRN=[]
            # time=lines[8][3:5]
            # print(time)
            # print(lines[HeaderLine])


            # 计算卫星的位置
            sat_pos=[]
            PRN_num=[]
            delta_t=[]
            for i in range(HeaderLine,len(lines)):
                # 获取时间数据
                year=int(lines[i][3:5].strip())
                if(year==time[0]):
                    month=int(lines[i][6:8].strip())
                    day=int(lines[i][9:11].strip())
                    hour=int (lines[i][12:14].strip())
                    minute=int(lines[i][15:17].strip())
                    sec= float(lines[i][18:22].strip())

                    # 读取卫星PRN号
                    PRN_number=lines[i][0:2].strip()
                    if(month==time[1]and day==time[2]and hour==time[3]and minute==time[4]and sec==time[5]):
                        # 读取钟差，钟漂之类的三个参数
                        time_change=[]
                        a=float(lines[i][22:37].strip())*pow(10,int(lines[i][38:41]))
                        time_change.append(a)
                        b=float(lines[i][41:56].strip())*pow(10,int(lines[i][57:60]))
                        time_change.append(b)
                        c=float(lines[i][60:75].strip())*pow(10,int(lines[i][76:79]))
                        time_change.append(c)

                        rows = 6
                        cols = 4
                        matrix = np.zeros((rows, cols))
                        # 读取卫星的相关参数矩阵
                        for j in range(0,6):
                          for k in range(0,4):
                            matrix[j][k]=float(lines[i+1+j][3+k*19:18+k*19].strip())*pow(10,int(lines[i+1+j][19+k*19:22+k*19]))
                        # 计算卫星的位置
                        [sat_x,sat_y,sat_z,delta_t_value]=caculate_pos_of_sat(matrix=matrix,a3=time_change,obstime=obstime,reftime=time)
                        PRN_num.append(PRN_number)
                        sat_pos.append([sat_x,sat_y,sat_z])
                        delta_t.append(delta_t_value)
                else:
                    continue

            return PRN_num,sat_pos,obs_type,delta_t
    

In [120]:

import numpy as np
# 定义需要进行计算的观测时刻
origin_obs_time=[23,11,30,0,0,0]
# 指定文件路径
obs_file_path = '../data/abpo3340.23o'

# 读取观测值文件
with open(obs_file_path, 'r') as file:
    obs_lines = file.readlines()

# 读取粗略坐标
approx_pos=[]
approx_pos_commmet='APPROX POSITION XYZ'
for i, line in enumerate(obs_lines, start=1):
    if approx_pos_commmet in line:
        approx_x=float(line[0:15].strip())
        approx_y=float(line[15:28].strip())
        approx_z=float(line[28:42].strip())
        approx_pos.append(approx_x)
        approx_pos.append(approx_y)
        approx_pos.append(approx_z)

obs_target_string='END OF HEADER'

obs_HeaderLine=0
for i, line in enumerate(obs_lines, start=1):
    if obs_target_string in line:
        obs_HeaderLine=i
        break


obs=[]
pos=[]
delta_t=[]
# 读取卫星编号
for i in range(obs_HeaderLine, len(obs_lines)):
        year=obs_lines[i][1:3]
        if(year=='23'):
            # 获取观测值的时间
            obs_time=[None]*6
            # str=obs_lines[i]
            obs_time[0]=int(obs_lines[i][1:3].strip())

            obs_time[1]=int(obs_lines[i][4:6].strip())
            obs_time[2]=int(obs_lines[i][7:9].strip())
            obs_time[3]=int(obs_lines[i][10:12].strip())
            obs_time[4]=int(obs_lines[i][13:15].strip())
            obs_time[5]=int(obs_lines[i][17:18].strip())

            if(obs_time!=origin_obs_time):
                break

            PRN_num,sat_pos,obs_type,delta_t_value=caculate_sat(obstime=obs_time)
            # 获取卫星的观测值的数目
            num_sat=int(obs_lines[i][30:32])
            if(num_sat%12==0):
                n=int(num_sat/12)
            else:
                n=int(num_sat/12)+1



            str=''
            for j in range(n):
                str=str+obs_lines[i+j][32:68].strip()
            
            obs_sat_PRN=[]
            for k in range(num_sat):
                obs_sat_PRN.append(str[k*3:k*3+3])


            i=i+n
            for index, indexPRN in enumerate(obs_sat_PRN):
                if(indexPRN[0]==obs_type[0]):
                    for index1,PRN_num1 in enumerate(PRN_num):
                        if(int(indexPRN[1:3])==int(PRN_num1)):
                            P=float(obs_lines[2*index+i][34:46].strip())
                            obs.append(P)
                            pos.append(sat_pos[index1])
                            delta_t.append(delta_t_value[index1])
        else:
            continue



In [95]:
import math

# 测试卫星坐标数据是否计算正确
def caculate_distance(point1,point2):
    distance=math.sqrt((point1[0]-point2[0])**2+(point1[1]-point2[1])**2+(point1[2]-point2[2])**2)
    return distance

# 将list数据转化成数组
approx_pos=np.array(approx_pos)
print(obs[1])

print(caculate_distance(pos[1],approx_pos))

23199615.854
23075680.62474096


# GNSS伪距单点定位的主要原理
如果通过广播星历以及观测值文件进行单点定位比较困难，需要熟系观测值文件格式以及卫星位置计算的一些算法。但是就比赛而言，核心的就只有误差方程的建立。因为比赛会直接给你卫星的坐标，GPS时，电离层，对流层延迟就可以。直接根据误差方程构建间接平差模型就可以。


## 对于每一个观测伪距

$$
R=\sqrt{\left( x_s-x_r \right) ^2+\left( y_s-y_r \right) ^2+\left( z_s-z_r \right) ^2}=f\left( x_r,y_r,z_r \right) 
$$

## 线性化（泰勒公式）：

$$
R^0+V=\sqrt{\left( x_s-x_{r}^{0} \right) ^2+\left( y_s-y_{r}^{0} \right) ^2+\left( z_s-z_{r}^{0} \right) ^2}+\frac{\partial f}{\partial x_r}\varDelta x_r+\frac{\partial f}{\partial y_r}\varDelta y_r+\frac{\partial f}{\partial z_r}\varDelta z_r
$$

## 上述公式线性化后可以化简为：
$$
V=\left( \frac{-\left( x_s-x_r \right)}{\rho _0},\frac{-\left( y_s-y_r \right)}{\rho _0},\frac{-\left( z_s-z_r \right)}{\rho _0},-c \right) \left[ \begin{array}{c}
	\varDelta x_r\\
	\varDelta y_r\\
	\varDelta z_r\\
	\sigma _{tr}\\
\end{array} \right] -L
$$
$$
L=\sigma _{ts}+R^0-\rho _0-Delay
$$
$$
\rho _0=\sqrt{\left( x_s-x_{r}^{0} \right) ^2+\left( y_s-y_{r}^{0} \right) ^2+\left( z_s-z_{r}^{0} \right) ^2}
$$

## 上述公式可以简化为
$$
V=Bx-L
$$

$$
x=\left( B^TB \right) ^{-1}\left( B^TL \right) 
$$

## 需要进行迭代的话迭代方程为
$$
x_r=x_r+\varDelta x_r
$$
$$
y_r=y_r+\varDelta y_r
$$
$$
z_r=z_r+\varDelta z_r
$$
$$

$$


In [116]:
def caculate_pos(approx,obs,pos):

    for k in range(50):
            # 记录初始数据
            x0=approx[0]
            y0=approx[1]
            z0=approx[2]
            # 误差方程中的B
            c=3*pow(10,8)
            
            B = np.zeros((len(obs), 4), dtype=float)
            L=np.zeros((len(obs),1),dtype=float)
            for i in range(len(obs)):
                        p0=caculate_distance(approx_pos,pos[i])
                        B[i,0]=-(pos[i][0]-approx[0])/p0
                        B[i,1]=-(pos[i][1]-approx[1])/p0
                        B[i,2]=-(pos[i][2]-approx[2])/p0
                        B[i,3]=-c

                        # 构建误差方程中的L
                        L[i,0]=delta_t[i]+obs[i]-p0
            
            btb=np.dot(np.transpose(B),B)
            a=np.linalg.det(btb)
            if(a==0):
                print("矩阵不可逆，定位失败，结束！")
                break
            btl=np.dot(np.transpose(B),L)
            x= np.dot(np.linalg.inv(btb),btl)

            # 迭代
            approx[0]=x[0][0]
            approx[1]=x[1][0]
            approx[2]=x[2][0]

            if(abs(approx[0]-x0)<5 and abs(approx[1]-x0)<5 and abs(approx[2]-x0)<5):
                   break
    return approx
    


In [121]:
if(len(obs<4)):
    print('不可以求解。多余观测值不够')

position=caculate_pos(approx_pos,obs,pos)

LinAlgError: Singular matrix